In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import pylab as pl
import numpy as np
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
from sklearn.metrics import r2_score
import joblib
from sklearn.preprocessing import StandardScaler

In [2]:
df=pd.read_csv("../Data/HouseImformation.csv")

<h3 id="">In Multiple Regression Model We found that the best method is to remove outliers. </h3>

In [3]:
df['Area'] = df['Area'].str.replace(',', '').astype(float)

df=df.dropna()

isoOuter=IsolationForest(contamination=0.01)
df['OuterData']=isoOuter.fit_predict(df[["Area", "Price"]])

df=df.drop(index=df[df['OuterData'] == -1].index)

MeanPriceAddressOuter=df.groupby("Address")["Price"].mean()
df["MeanPriceA"] = df["Address"].map(MeanPriceAddressOuter)

In [4]:
mask= np.random.rand(len(df)) < 0.8

train=df[mask]
test=df[~mask]

x_train=np.asanyarray(train[['Area','Room','Parking','Warehouse','Elevator','MeanPriceA']])
y_train=np.asanyarray(train[['Price']])

x_test=np.asanyarray(test[['Area','Room','Parking','Warehouse','Elevator','MeanPriceA']])
y_test=np.asanyarray(test[['Price']])


scaler=StandardScaler()

x_train_scaler=scaler.fit_transform(x_train)

x_test_scaler=scaler.fit_transform(x_test)


<h3 id="">Find The Best Alpha</h3>

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

ridge=Ridge()

alpha_Params={'alpha' : [0.001,0.01,0.1,1.0,10.0,100.0]}

grid=GridSearchCV(ridge,alpha_Params,cv=5)

grid.fit(x_train_scaler,y_train)

print("Best alpha:", grid.best_params_)
print("Best CV score:", grid.best_score_)



y_predict = grid.predict(x_test_scaler)

print("R2-score: %.2f" % r2_score(y_test , y_predict))


Best alpha: {'alpha': 100.0}
Best CV score: 0.7287397658012796
R2-score: 0.74


In [6]:
joblib.dump(grid, '../Models/RidgeRegressionModel.pkl')

['../Models/RidgeRegressionModel.pkl']